# 가우시안 혼합

가우시안 혼합 모델(Gaussian Mixture Model, GMM)은 여러 개의 혼합된 가우시안 분포에서 샘플이 생성되었다고 가정하는 확률 모델을 의미한다. 

현실에 존재하는 복잡한 형태의 확률 분포를 k개의 가우시안 분포를 혼합해 표현하는 것이 GMM의 기본 아이디어이다.(각 가우시안 분포는 하나의 클러스터를 의미, 즉 k개의 클러스터)

GMM에서 $x$가 발생할 확률은 다음과 같이 여러 가우시안 분포 밀도 함수의 합으로 표현한다.

$$p(x) = \sum^K_{k=1} \pi_k N(x | \mu_k,\textstyle \sum_k)$$

여기서, $\pi_k$는 $k$번째 가우시안 분포가 선택될 확률을 나타내며, 다음의 조건을 만족한다.

$$0 \le \pi_k \le 1$$
$$\sum^K_{k=1}\pi_k = 1$$

따라서, GMM을 학습시킨다는 것은 주어진 데이터 $X={x_1, x_2, \dots, x_N}$에 대한 $\pi_k$, $\mu_k$, $\sum_k$를 추정하는 것과 같다.